
# OZON RecSys Baseline - Рекомендательная система для категории Apparel
Этот ноутбук содержит базовое решение для задачи предсказания следующей покупки пользователя в категории одежды, обуви и аксессуаров.

## Задача
- Предсказать топ-100 товаров для каждого пользователя из тестовой выборки
- Метрика оценки: NDCG@100
- Данные: ~38GB в формате parquet, 1.6B взаимодействий, 19M заказов


In [1]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from collections import defaultdict
import subprocess
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import dask.dataframe as dd

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Загрузка и анализ данных


In [2]:
def load_train_data():
    print("Загружаем тренировочные данные через Dask...")

    orders_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_orders_data/*/*.parquet'
    tracker_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_tracker_data/*/*.parquet'
    items_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train/final_apparel_items_data/*.parquet'
    categories_path = '/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_train_final_categories_tree/*.parquet'

    # --- Загружаем заказы ---
    orders_ddf = dd.read_parquet(orders_path)
    print(f"Найдено файлов заказов: {orders_ddf.npartitions} частей")

    # --- Загружаем взаимодействия ---
    tracker_ddf = dd.read_parquet(tracker_path)
    print(f"Найдено файлов взаимодействий: {tracker_ddf.npartitions} частей")

    # --- Загружаем товары ---
    items_ddf = dd.read_parquet(items_path)
    print(f"Найдено файлов товаров: {items_ddf.npartitions} частей")

    # --- Загружаем категории ---
    filter_ids = {7500, 7697, 17777}
    categories_ddf = dd.read_parquet(categories_path)

    def filter_ids_len5(df):
        mask = df['ids'].apply(lambda x: bool(set(x) & filter_ids) and len(x) == 5)
        return df[mask]

    categories_ddf = categories_ddf.map_partitions(filter_ids_len5)

    print(f"Категорий после фильтрации: {categories_ddf.shape[0].compute():,}")

    # --- Возвращаем Dask DataFrame ---
    return orders_ddf, tracker_ddf, items_ddf, categories_ddf

In [3]:
orders_df, tracker_df, items_df, category_df = load_train_data()

Загружаем тренировочные данные через Dask...
Найдено файлов заказов: 28 частей
Найдено файлов взаимодействий: 298 частей
Найдено файлов товаров: 100 частей
Категорий после фильтрации: 291


In [13]:
orders_df.head(5)

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
0,7510328,981,2025-07-02 08:27:17.686,delivered_orders,2025-07-06 16:14:12,2025-07-02
1,136809690,4670,2025-07-02 17:17:30.423,canceled_orders,2025-07-06 19:35:07,2025-07-02
2,232878073,12341,2025-07-02 15:17:46.393,canceled_orders,2025-07-11 15:36:40,2025-07-02
3,42322137,12860,2025-07-02 00:40:52.503,delivered_orders,2025-07-04 19:44:48,2025-07-02
4,22480781,15330,2025-07-02 07:25:11.226,delivered_orders,2025-07-03 20:07:33,2025-07-02


In [14]:
tracker_df.head(5)

,item_id,user_id,timestamp,action_type,action_widget,date
0,678602,4211601,2025-05-05 18:11:15,page_view,cart.cartSplit,2025-05-05
1,678602,4211601,2025-05-05 18:12:57,view_description,pdp.characteristics,2025-05-05
2,678602,4211601,2025-05-05 18:12:52,view_description,pdp.richContent,2025-05-05
3,678602,4211601,2025-05-05 10:39:08,page_view,cart.cartSplit,2025-05-05
4,678602,4211601,2025-05-05 18:12:57,view_description,pdp.characteristics,2025-05-05


In [15]:
items_df.head(5)

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
0,400301,Очки солнцезащитные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080,243954236,33306429
1,678602,Футболка GOLDUSTIM Футболка женская,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508,8039633,27057398
2,746401,Худи Patagonia,"[{'attribute_name': 'Type', 'attribute_value':...","[0.47452784, -0.4727797, -0.51971936, -0.08221...",7555,120078071,11645051
3,872829,"Комплект носков adidas 3S C Spw Mid 3P, 3 пары","[{'attribute_name': 'Type', 'attribute_value':...","[-0.06987186, -2.2802832, -0.5829071, -0.33674...",36563,247120069,38088455
4,893255,Жилет Buy & Style жилеты,"[{'attribute_name': 'Type', 'attribute_value':...","[0.12488763, 0.5466293, 0.29031003, 0.21499962...",7535,327822613,19339929


In [16]:
category_df.head(5)

,catalogid,catalogpath,ids
275,7534,"[{'id': 7534, 'name': 'Дубленки и шубы', 'full...","[7534, 7528, 7501, 7500, -1]"
276,7529,"[{'id': 7529, 'name': 'Жилеты утепленные', 'fu...","[7529, 7528, 7501, 7500, -1]"
277,34590,"[{'id': 34590, 'name': 'Комплекты верхней одеж...","[34590, 7528, 7501, 7500, -1]"
278,7530,"[{'id': 7530, 'name': 'Куртки и пуховики', 'fu...","[7530, 7528, 7501, 7500, -1]"
279,7531,"[{'id': 7531, 'name': 'Пальто', 'fullName': 'П...","[7531, 7528, 7501, 7500, -1]"


In [17]:
# items_df
items_df_apparel = items_df[items_df['catalogid'].isin(category_df['catalogid'])]
items_df_apparel.head(5)

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
0,400301,Очки солнцезащитные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080,243954236,33306429
1,678602,Футболка GOLDUSTIM Футболка женская,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508,8039633,27057398
8,1841732,Сумка женская через плечо кросс боди маленькая...,"[{'attribute_name': 'Type', 'attribute_value':...","[0.845032, -1.2555618, -0.15746951, 0.296685, ...",17002,273455755,12361995
10,2095808,Футболка Tom Tailor,"[{'attribute_name': 'Type', 'attribute_value':...","[0.032773145, -0.75285774, -0.3085317, 0.00191...",7559,289182638,21889175
11,2141800,Футболка,"[{'attribute_name': 'Type', 'attribute_value':...","[0.23843573, -0.773998, -1.0875425, -0.4103259...",7559,238432973,15795237


## 2. Exploratory Data Analysis (EDA)


In [9]:
print("АНАЛИЗ ЗАКАЗОВ")
print("=" * 50)
print(f"Общее количество заказов: {len(orders_df):,}")
print(f"Уникальных пользователей: {orders_df['user_id'].nunique():,}")
print(f"Уникальных товаров: {orders_df['item_id'].nunique():,}")

# Если есть колонка created_date
if 'created_date' in orders_df.columns:
    print(f"Период данных: {orders_df['created_date'].min()} - {orders_df['created_date'].max()}")

print("\nРаспределение статусов заказов:")
status_counts = orders_df['last_status'].value_counts()

for status, count in status_counts.items():
    percentage = count / len(orders_df) * 100
    print(f"  {status}: {count:,} ({percentage:.1f}%)")

АНАЛИЗ ЗАКАЗОВ
Общее количество заказов: 1,475,216


TypeError: unsupported format string passed to Scalar.__format__

In [ ]:
print("\nАНАЛИЗ ВЗАИМОДЕЙСТВИЙ")
print("=" * 50)
print(f"Общее количество взаимодействий: {len(tracker_df):,}")
print(f"Уникальных пользователей: {tracker_df['user_id'].nunique():,}")
print(f"Уникальных товаров: {tracker_df['item_id'].nunique():,}")

print("\nРаспределение типов действий:")
action_counts = tracker_df['action_widget'].value_counts()
for action, count in action_counts.items():
    percentage = count / len(tracker_df) * 100
    print(f"  {action}: {count:,} ({percentage:.1f}%)")

In [ ]:
print("\nАНАЛИЗ ТОВАРОВ")
print("=" * 50)
print(f"Общее количество товаров: {len(items_df_apparel):,}")
print(f"Товары с эмбеддингами: {items_df_apparel['fclip_embed'].notna().sum():,}")

if 'attributes' in items_df_apparel.columns:
    print(f"Товары с атрибутами: {items_df_apparel['attributes'].notna().sum():,}")

print("\nТоп-10 категорий:")
top_categories = items_df_apparel['catalogid'].value_counts().head(10)
for cat_id, count in top_categories.items():
    print(f"  Категория {cat_id}: {count:,}")

## 3. Загрузка тестовых пользователей

In [ ]:
def load_test_users():

    print("Загружаем тестовых пользователей...")
    
    test_files = glob.glob('/home/root6/python/e_cup/rec_system/data/raw/ml_ozon_recsys_test_for_participants/test_for_participants/*.parquet')
    all_users = set()
    
    for file_path in tqdm(test_files, desc="Обработка тестовых файлов"):
        df = pd.read_parquet(file_path)
        all_users.update(df['user_id'].unique())
    
    print(f"Найдено уникальных тестовых пользователей: {len(all_users):,}")
    return list(all_users)

test_users = load_test_users()

In [ ]:
test_users[:5]

## 4. Построение модели